# Explore data files on local disk

In [ ]:
import pandas as pd
import numpy

In [ ]:
dataroot = "/data/data-files/graph-data/northwind/data"
orders = pd.read_csv(dataroot + "/orders.csv")
order_details = pd.read_csv(dataroot + "/order-details.csv")
customers = pd.read_csv(dataroot + "/customers.csv")
reps = pd.read_csv(dataroot + "/employees.csv")

In [ ]:
order_details.dtypes


In [ ]:
order_details.describe().round()

In [ ]:
order_details.info()

In [ ]:
order_details.head(20)

In [ ]:
customers['Region'].unique()

In [ ]:
%%bash
# rm -fv orders.db

In [ ]:
orders.info()

In [ ]:
orders[orders['CommissionAmt'].notnull()]

In [ ]:
# conn = sqlite3.connect('orders.db')
# orders.to_sql('orders', conn, if_exists='replace')
# order_details.to_sql('order_details', conn, if_exists='replace')
# conn.close()

In [ ]:
reps['BirthDate']

# Connect to the Northwind Graph using PyTigerGraph

In [ ]:
# import sqlite3
import pyTigerGraph as tg

In [ ]:
# host = "http://medsales.34.106.148.222.nip.io:14240"
host       = "https://homelab-k3s.172.16.17.201.nip.io" # must include the protocol http or https
username   = "tigergraph"
password   = "Tigergraph"
graphName  = "Northwind" # leave blank to use Global
restppPort = 9000       # default 9000
gsPort     = 14240       # default 14240
mysecret   = "ll5k45jrrvu0b4gmk4f7fokilgbgam4g"
# First establish a basic connection using a secret.  Do *not* do this if you already have a token
# host = "http://34.106.148.222.nip.io"
# token="o4luvshk47rticuhqi0cdokiq739o8i9"
conn = tg.TigerGraphConnection(host=host, restppPort=restppPort, gsPort=gsPort, graphname=graphName, password=password)
token = conn.getToken(mysecret, setToken=True, lifetime=None)

# Next use the new token to establish a full access connection for use with GSQL

conn = tg.TigerGraphConnection(host=host, restppPort=restppPort, gsPort=gsPort, graphname=graphName, password=password, apiToken=token[0])


In [ ]:
token[0]

In [ ]:
conn.getVertexTypes()

In [ ]:
# conn.getVertexTypes()
# conn.runInstalledQuery('floors')
print(conn.gsql('show edge *', options=[]))

In [ ]:
conn.gsql('drop graph gort', options=[])

In [ ]:
print(conn.gsql('show graph *', options=[]))

# Reps and customers